Interactive maps on Leaflet

In [29]:
import folium
from folium.plugins import MarkerCluster

In [6]:
m = folium.Map(location=[60.25, 24.8], tiles='Stamen Toner',
                   zoom_start=10, control_scale=True)

In [7]:
outfp = "./base_map1.html"

In [8]:
m.save(outfp)

In [9]:
 m2 = folium.Map(location=[40.730610, -73.935242], tiles='Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True)

outfp2 = "./base_map2.html"
    
# Save the map
 m2.save(outfp2)

In [18]:
import geopandas as gpd
from fiona import *

fp = "dataE5/dataE5/Vaestotietoruudukko_2015.shp"
addr_fp = "dataE5/dataE5/addresses.shp"

In [13]:
data = gpd.read_file(fp)
ad = gpd.read_file(addr_fp)

In [19]:
data['geometry'] = data['geometry'].to_crs(epsg=4326)
ad['geometry'] = ad['geometry'].to_crs(epsg=4326)

In [24]:
data = data.loc[(data['ASUKKAITA'] > 0) & (data['ASUKKAITA'] <= 1000)]

In [25]:
data['geoid'] = data.index.astype(str)
ad['geoid'] = ad.index.astype(str)

In [26]:
data = data[['geoid', 'ASUKKAITA', 'geometry']]

In [27]:
jsontxt = data.to_json()

In [31]:
marker_cluster = MarkerCluster().add_to(m)

In [33]:
m.choropleth(geo_data=jsontxt, data=data, columns=['geoid', 'ASUKKAITA'], key_on="feature.id",
                   fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.2, line_color='white', line_weight=0,
                   threshold_scale=[100, 250, 500, 1000, 2000],
                   legend_name='Population in Helsinki', highlight=False, smooth_factor=1.0)


In [34]:
for idx, row in ad.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get address information
    address = row['address']
    # Add marker to the map
    folium.RegularPolygonMarker(location=[lat, lon], popup=address, fill_color='#2b8cbe', number_of_sides=6, radius=8).add_to(marker_cluster)


In [36]:
outfp = r'./pop15.html'
m.save(outfp)